In [58]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

import catboost as cb

In [59]:
# Load files
train = pd.read_csv('New_Train.csv')
test = pd.read_csv('New_Test.csv')
SampleSubmission = pd.read_csv('SampleSubmission.csv')

In [60]:
# identify target
target_cols = []
for i in train.columns.values:
    if i not in test.columns.values:
        target_cols.append(i)
        
target = train[target_cols]
train[target_cols].head()


,target
0,51.500000
1,55.869999
2,47.520000
3,58.599998
4,76.599998


In [61]:
train

,child_age,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,child_observe_concentrated,child_observe_diligent,child_observe_interested,child_observe_total,...,pri_support,pri_support_providerother,pri_support_frequency,pri_fees_amount_4_6,pri_internet_user,pri_fees_amount_pv,pri_funding_subsidy,pri_meals,pri_languages,target
0,59.000000,NaN,NaN,NaN,NaN,Sometimes,Sometimes,Sometimes,Sometimes,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.500000
1,60.163933,NaN,NaN,1st year in the programme,103.000000,Sometimes,Almost never,Sometimes,Often,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.869999
2,69.000000,NaN,NaN,NaN,108.400002,Often,Often,Sometimes,Often,7.0,...,NaN,NaN,NaN,NaN,1 2 3 4,0.00000,NaN,NaN,NaN,47.520000
3,53.000000,20.0,No,1st year in the programme,98.099998,Almost always,Almost always,Sometimes,Often,9.0,...,NaN,NaN,NaN,NaN,NaN,373.65045,0.0,Yes,NaN,58.599998
4,57.000000,0.0,NaN,2nd year in programme,114.000000,Almost always,Almost always,Almost always,Almost always,12.0,...,Yes,NaN,Once a month,NaN,NaN,106.75727,0.0,Yes,NaN,76.599998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8580,55.000000,9.0,Yes,1st year in the programme,102.300003,Often,Often,Sometimes,Sometimes,6.0,...,Yes,NaN,Once a month,NaN,NaN,181.48737,1.0,Yes,NaN,23.469999
8581,55.000000,32.0,NaN,2nd year in programme,102.599998,Often,Sometimes,Sometimes,Almost never,4.0,...,No,NaN,NaN,NaN,NaN,213.51454,1.0,Yes,NaN,45.639999
8582,56.000000,45.0,NaN,3rd year in programme,103.800003,Almost always,Almost always,Almost always,Almost always,12.0,...,Yes,NaN,Once a year,NaN,NaN,960.81540,0.0,Yes,NaN,53.290001
8583,57.000000,9.0,Yes,1st year in the programme,102.400002,Almost always,Almost always,Almost always,Almost always,12.0,...,NaN,NaN,NaN,NaN,NaN,0.00000,1.0,Yes,NaN,56.330002


### helper functions

In [62]:
# Helper functions

def group_by_name(df, name, starts_with = False):
    arr = []
    if starts_with:
        for col in df.columns:
            if col.startswith(name):
                arr.append(col)   
    else:
        for col in df.columns:
            if name in col:
                arr.append(col)
        
            
    return arr

def analyse_group(df):
    features = []; cat_features = []; not_features = []
    for k in df.columns:
        features.append(k)
        if df[k].dtype == 'O':
            cat_features.append(k)
            print('There are '+ str(len(df[k].value_counts()))+' Classes in: ' +k)
            print('They are '+ str(df[k].unique()))
        else:
            not_features.append(k)

    print('----------------------------------')
    print('We have '+str(len(features)) + ' features')
    print('We have '+str(len(cat_features)) + ' categorical features')
    print('We have '+str(len(not_features)) + ' numerical features')

def count_class(data):
    for col in data.columns:
        if data[col].dtype == 'O':
            print('There are '+ str(len(data[col].value_counts()))+' Classes in: ' +col)
            

## Preprocessing

In [63]:
# select features according to correlatin to target
train['target'] = target
# get the correlation of each feature with respect to the target (emmission)
train_corr = train.corr()['target'].abs().sort_values(ascending=False)

# get features with a correlation above a certain threshold
features = train_corr[train_corr > 0].index.to_list()[1:]

print(len(features))

train = train[features]
test  = test[features]

# train['target'] = target
# train.drop(['target'],inplace=True,axis=1)


116


In [64]:
train.dtypes

child_observe_total              float64
child_age                        float64
child_height                     float64
pri_fees_amount_2_3              float64
pri_fees_amount                  float64
                                  ...   
count_register_year_2019         float64
count_register_year_2017         float64
pra_class_attendance_precovid    float64
pri_days                         float64
pri_difficult_walk               float64
Length: 116, dtype: object

In [65]:
# train.drop(['child_id'],inplace=True,axis=1)
# test.drop(['child_id'],inplace=True,axis=1)

In [66]:
features = []; cat_features = []; not_features = []
for k in train.columns:
    features.append(k)
    if train[k].dtype == 'O':
        cat_features.append(k)
#         print('There are '+ str(len(df[k].value_counts()))+' Classes in: ' +k)
#         print('They are '+ str(df[k].unique()))
    else:
        not_features.append(k)

print('----------------------------------')
print('We have '+str(len(features)) + ' features')
print('We have '+str(len(cat_features)) + ' categorical features')
print('We have '+str(len(not_features)) + ' numerical features')


----------------------------------
We have 116 features
We have 0 categorical features
We have 116 numerical features


In [67]:
features

['child_observe_total',
 'child_age',
 'child_height',
 'pri_fees_amount_2_3',
 'pri_fees_amount',
 'pri_fees_amount_0_1',
 'pri_fees_amount_pv',
 'count_toilets_children',
 'pri_amount_funding_fees',
 'count_staff_qual_nqf6_9',
 'child_zha',
 'count_staff_contract_permanent',
 'count_staff_paid',
 'count_staff_gender_female',
 'pri_expense_staff',
 'count_staff_paid_assistants',
 'count_staff_all',
 'count_staff_time',
 'count_staff_gender',
 'count_staff_salary',
 'count_staff_contract',
 'pri_expense_admin',
 'count_staff_salary_paid',
 'count_staff_time_full',
 'count_staff_paid_practitioners',
 'count_register_race_white',
 'count_present_girls',
 'pri_fees_amount_4_6',
 'count_children_present',
 'count_staff_paid_support',
 'count_present_boys',
 'count_register_year_2015',
 'pqa_score_teaching',
 'count_register_race_indian',
 'pri_expense_rent',
 'pri_time_close_hours',
 'count_practitioners_age_0',
 'pri_expense_maintenance',
 'count_children_attendance',
 'pri_funding_subsid

In [68]:
train[cat_features] = train[cat_features].astype(str) 
test[cat_features] = test[cat_features].astype(str) 

train[not_features] = train[not_features].fillna(0)
test[not_features] = test[not_features].fillna(0)

# train = train[not_features].astype(np.int) 
# test= test[not_features].astype(np.int) 

# train = train.fillna('')
# test  = test.fillna('')

In [69]:
# y = train.target
X_train, X_test, y_train, y_test = train_test_split(
    train,
    target,
    test_size = 0.15,
    random_state = 42,
    shuffle = True
)

In [70]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7297, 116), (1288, 116), (7297, 1), (1288, 1))

In [71]:
# cb.Pool?

In [72]:
# y_test = test[features[:-1]]
train_dataset = cb.Pool(data = X_train, label = y_train, cat_features=cat_features)
test_dataset = cb.Pool(data = X_test, label = y_test, cat_features=cat_features)
model = cb.CatBoostRegressor(iterations = 1153, learning_rate=0.1, random_seed=123, verbose=300)
model.fit(train_dataset, eval_set=test_dataset, use_best_model=True, early_stopping_rounds=300)
y_pred = model.predict(X_test)
# preds_test  = model.predict(test)
print(np.sqrt(mean_squared_error(y_test, y_pred)))
# 9.847991796804552

0:	learn: 14.6755355	test: 15.1398521	best: 15.1398521 (0)	total: 14.2ms	remaining: 16.3s
300:	learn: 9.8814529	test: 11.4110886	best: 11.4052364 (296)	total: 1.84s	remaining: 5.22s
600:	learn: 8.9385575	test: 11.3509947	best: 11.3500277 (518)	total: 3.62s	remaining: 3.32s
900:	learn: 8.2687569	test: 11.3489725	best: 11.3360979 (774)	total: 5.13s	remaining: 1.44s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 11.33609789
bestIteration = 774

Shrink model to first 775 iterations.
11.33609789044558


In [73]:
import shap
expaliner = shap.TreeExplainer(model)
shap_values = expaliner.shap_values(test)

In [74]:
f = {
    'f1' : [], 'f2': [], 'f3': [], 'f4': [],'f5': [],
    'f6' : [], 'f7': [], 'f8': [], 'f9': [],'f10': [],
    'f11' : [], 'f12': [], 'f13': [], 'f14': [],'f15': []
}

In [75]:
for shap_value in shap_values:
    arr = np.argsort(shap_value)[::-1][:15]
    for ind, a in enumerate(arr):
        name_f = f'f{ind+1}'
        f[name_f].append(features[a])

In [76]:
# Make predictions on the test set
pred = model.predict(test)
pred

array([57.75689296, 43.24009713, 46.98626818, ..., 49.7134964 ,
       42.74160993, 33.06936191])

In [77]:
SampleSubmission['target']    = pred
SampleSubmission['feature_1'] = f['f1']
SampleSubmission['feature_2'] = f['f2']
SampleSubmission['feature_3'] = f['f3']
SampleSubmission['feature_4'] = f['f4']
SampleSubmission['feature_5'] = f['f5']
SampleSubmission['feature_6'] = f['f6']
SampleSubmission['feature_7'] = f['f7']
SampleSubmission['feature_8'] = f['f8']
SampleSubmission['feature_9'] = f['f9']
SampleSubmission['feature_10'] = f['f10']
SampleSubmission['feature_11'] = f['f11']
SampleSubmission['feature_12'] = f['f12']
SampleSubmission['feature_13'] = f['f13']
SampleSubmission['feature_14'] = f['f14']
SampleSubmission['feature_15'] = f['f15']

In [78]:
SampleSubmission

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,57.756893,child_observe_total,count_children_present,pri_fees_amount_pv,count_register_year_2016,pri_fees_amount,pri_time_open_hours,count_register_gender_female,count_staff_qual_skills,count_register_year_2017,count_register_gender_male,pri_dsd_year,count_register_year_2018,count_register_year_grader,count_staff_time,child_zha
1,ID_GQ6ONJ4FP,43.240097,child_observe_total,count_register_year_2015,pri_fees_amount,count_children_present,pri_fees_amount_pv,count_staff_qual_nqf4_5,count_children_precovid,count_register_year_2018,pri_amount_funding_dsd,count_register_year_grader,count_toilets_children,pra_class_size,pra_class_space,pri_fees_amount_4_6,count_staff_salary_paid
2,ID_YZ76CVRW3,46.986268,child_observe_total,pri_fees_amount_pv,count_register_year_2019,count_register_all,count_register_year_2021,pri_fees_amount,pri_funding_subsidy,count_register_year_2017,count_register_year_2020,count_register_year_grader,count_staff_time_full,count_staff_qual_nqf4_5,pri_time_open_hours,count_register_race_african,count_staff_time
3,ID_BNINCRXH8,64.305064,child_observe_total,count_children_attendance,child_age,count_children_present,child_attendance,pri_fees_amount_pv,pri_capacity,child_height,child_zha,count_toilets_children,pri_funding_subsidy,count_register_gender_female,count_staff_gender_female,count_register_year_grader,count_register_year_2020
4,ID_1U7GDTLRI,42.046062,child_observe_total,count_register_year_2016,pri_funding_subsidy,count_children_precovid,count_register_year_2020,pri_year,child_months_enrolment,count_register_year_grader,pri_dsd_year,count_staff_qual_nqf4_5,count_staff_time_full,count_staff_time,count_register_gender_female,pri_fees_amount_0_1,count_register_gender_male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,ID_LBPQ2VMQZ,56.244167,child_age,child_observe_total,count_children_attendance,pri_time_open_hours,pri_days,pri_funding_subsidy,count_register_year_2017,count_toilets_children,count_register_all,count_register_year_2020,child_zha,count_staff_time_full,pri_amount_funding_dsd,obs_classrooms,count_register_race_african
3676,ID_H2RKKMMKK,47.696626,child_age,pri_fees_amount,count_children_present,child_height,count_register_year_2020,count_register_year_2019,count_register_year_grader,count_register_year_2017,count_register_gender_male,count_register_year_2016,count_register_race,child_zha,count_register_gender_female,pri_dsd_year,count_register_race_coloured
3677,ID_VY8KX7YTZ,49.713496,child_observe_total,child_age,count_register_race_african,pra_class_space,pri_expense_materials,pri_clinic_time,count_staff_qual_nqf4_5,count_practitioners_age_4,count_register_gender_male,count_staff_time_full,count_children_precovid,count_register_year_2020,pri_expense_food,pri_time_open_minutes,pri_year
3678,ID_EO2MYZ4M7,42.741610,child_zha,count_staff_salary_unpaid,pri_expense_admin,pri_expense_other,count_children_present,pqa_score_relationships,child_height,pri_expense_staff,count_register_year_2018,count_register_year_2019,pri_capacity,count_register_year_2020,count_children_precovid,count_register_gender_female,pri_expense_food


In [79]:
SampleSubmission.to_csv('submission.csv', index=False)
